In [157]:
# install required modules
!pip install pymongo
!pip install pandas

In [158]:
# access modules
from pandas import pandas as pd
from pymongo import MongoClient

In [159]:
mongo_client = MongoClient('34.209.155.42', 27016)
database_reference = mongo_client.twitter
collection_reference = database_reference.tweets

In [160]:
collection_reference.count()

2124309

In [161]:
# Get the first document
cursor = collection_reference.find()

In [162]:
sample_set = collection_reference.aggregate([{'$sample': {'size': 10}}])
df = pd.DataFrame(list(sample_set))
df.dtypes

_id                           object
contributors                  object
coordinates                   object
created_at                    object
display_text_range            object
entities                      object
extended_tweet                object
favorite_count                 int64
favorited                       bool
filter_level                  object
geo                           object
id                             int64
id_str                        object
in_reply_to_screen_name       object
in_reply_to_status_id        float64
in_reply_to_status_id_str     object
in_reply_to_user_id          float64
in_reply_to_user_id_str       object
is_quote_status                 bool
lang                          object
place                         object
possibly_sensitive            object
quote_count                    int64
quoted_status                 object
quoted_status_id             float64
quoted_status_id_str          object
reply_count                    int64
r

In [163]:
df.head(n=5)

,_id,contributors,coordinates,created_at,display_text_range,entities,extended_tweet,favorite_count,favorited,filter_level,...,quoted_status_id_str,reply_count,retweet_count,retweeted,source,text,timestamp_ms,truncated,user,user_processed
0,5a74edb215ba4c0001541974,None,None,Fri Feb 02 23:01:06 +0000 2018,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...","God, I hope Stan Lee’s cameo in Black Panther ...",1517612466386,False,"{'id': 426964605, 'id_str': '426964605', 'name...",NaN
1,5a74bb7036dd5f00016d507f,None,None,Fri Feb 02 19:26:40 +0000 2018,"[11, 22]","{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/#!/download/ipad"" ...",@joanna__r Habit 🤷🏻‍♂️,1517599600331,False,"{'id': 62141145, 'id_str': '62141145', 'name':...",NaN
2,5a74ee4615ba4c00015419ee,None,None,Fri Feb 02 23:03:34 +0000 2018,"[15, 51]","{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",@annagesselleg My interactions with 90% of boy...,1517612614614,False,"{'id': 1024548752, 'id_str': '1024548752', 'na...",NaN
3,5a66e02636dd5f000165219c,None,None,Tue Jan 23 07:11:33 +0000 2018,"[0, 44]","{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,0,False,low,...,955697147432882177,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",She has to be announcing this tour tomorrow. h...,1516691493906,False,"{'id': 342953499, 'id_str': '342953499', 'name...",NaN
4,5a51624336dd5f0001581cea,None,"{'type': 'Point', 'coordinates': [-118.243, 34...",Sat Jan 06 23:56:50 +0000 2018,NaN,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,0,False,low,...,NaN,0,0,False,"<a href=""http://instagram.com"" rel=""nofollow"">...",'you look so fine\nI'm like the desert tonight...,1515283010785,False,"{'id': 26154884, 'id_str': '26154884', 'name':...",NaN


In [164]:
# check out contributor
df.isnull().sum(axis=0)

_id                           0
contributors                 10
coordinates                   8
created_at                    0
display_text_range            4
entities                      0
extended_tweet                9
favorite_count                0
favorited                     0
filter_level                  0
geo                           8
id                            0
id_str                        0
in_reply_to_screen_name       6
in_reply_to_status_id         6
in_reply_to_status_id_str     6
in_reply_to_user_id           6
in_reply_to_user_id_str       6
is_quote_status               0
lang                          0
place                         0
possibly_sensitive            6
quote_count                   0
quoted_status                 8
quoted_status_id              8
quoted_status_id_str          8
reply_count                   0
retweet_count                 0
retweeted                     0
source                        0
text                          0
timestam

In [165]:
# I want to parse the source column, so I can gather details about the devices used to tweet
df.loc[1, 'source']

'<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>'

In [166]:
df['source'] = df['source'].str.replace('<.*?>', '')
df['source']

0     Twitter for iPhone
1       Twitter for iPad
2     Twitter for iPhone
3     Twitter for iPhone
4              Instagram
5     Twitter for iPhone
6     Twitter for iPhone
7    Twitter for Android
8              Instagram
9     Twitter for iPhone
Name: source, dtype: object

### Place contains location info we need. Coordinates and geo are incomplete/redundant cols that provide a subset of info place provides. So we can remove coordinates and geo

In [167]:
df.loc[1, 'coordinates']

In [168]:
df.loc[1, 'geo']

In [169]:
df.loc[1, 'place'].get('country')

'United States'

In [170]:
x = df['place'].get('country')
x